In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_diagonal_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,1888079.57,0.70,5.79,-7.39,0.02,-0.16,-0.02
4,0.70,1052609.25,-10.48,-5.87,2.73,0.14,0.02
5,5.79,-10.48,175510.94,0.64,0.92,0.35,-0.02
6,-7.39,-5.87,0.64,53930.97,0.04,-0.01,-0.00
7,0.02,2.73,0.92,0.04,6244.79,0.02,0.00
8,-0.16,0.14,0.35,-0.01,0.02,254.71,0.00
9,-0.02,0.02,-0.02,-0.00,0.00,0.00,5.92


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,0.00000,0.00001,-0.00002,0.00000,-0.00001,-0.00001
4,0.00000,1.00000,-0.00002,-0.00002,0.00003,0.00001,0.00001
5,0.00001,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,-0.00002,-0.00002,0.00001,1.00000,0.00000,-0.00000,-0.00001
7,0.00000,0.00003,0.00003,0.00000,1.00000,0.00001,0.00002
8,-0.00001,0.00001,0.00005,-0.00000,0.00001,1.00000,0.00001
9,-0.00001,0.00001,-0.00002,-0.00001,0.00002,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

21.988309010408777

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[5.15643367e+11 1.52232031e+11 4.12824882e+09 3.61720215e+08
 4.32770816e+06 5.40709588e+03 1.31190907e-29]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.991263,-0.131063,-0.014164,-0.004318,-0.000500,-0.000020,-4.836879e-07
1,0.130424,0.990728,-0.036435,-0.010856,-0.001250,-0.000051,-1.162258e-06
2,0.018254,0.033277,0.995667,-0.084449,-0.008639,-0.000350,-8.309003e-06
3,0.007226,0.012979,0.083530,0.995593,-0.039928,-0.001600,-3.730632e-05
4,0.001104,0.001978,0.011882,0.039000,0.998956,-0.020496,-4.747966e-04
5,0.000068,0.000121,0.000723,0.002360,0.020395,0.999518,-2.325668e-02
6,0.000003,0.000006,0.000034,0.000110,0.000947,0.023242,9.997294e-01


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.008736727313238823,
 0.009271658468567923,
 0.004332876636626293,
 0.004407058805397712,
 0.001044344426413657,
 0.00048159696544436414,
 0.0002705866166931026]